In [ ]:
import ee
import geemap
ee.Initialize()


In [ ]:
Map = geemap.Map()
Map.add_basemap('Esri.WorldStreetMap')
Map

In [ ]:
# shp转为gee几何对象
roi = geemap.shp_to_ee(r"E:\数据处理\区域\a\区域面积\区域面积融合.shp").geometry()
Map.addLayer(roi, {}, 'Boundary')
Map.center_object(roi)

In [ ]:
# 裁剪
#Map.draw_features
#feature = Map.draw_last_feature
#loc = feature.geometry()

In [ ]:
# The code has been copied to the clipboard. 
# Press Ctrl+V to in a code cell to paste it.
def maskL8sr(image):
    # Bit 0 - Fill
    # Bit 1 - Dilated Cloud
    # Bit 2 - Cirrus
    # Bit 3 - Cloud
    # Bit 4 - Cloud Shadow
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)

    # Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)

    # Replace the original bands with the scaled ones and apply the masks.
    return image.addBands(opticalBands, None, True) \
    .addBands(thermalBands, None, True) \
    .updateMask(qaMask) \
    .updateMask(saturationMask)

# 去除山体阴影
def hillshade(img, elevation):
  # 太阳方位角
  azimuth = img.getNumber('SUN_AZIMUTH')
  # 太阳天顶角=90-太阳仰角
  zenith = ee.Number(90).subtract(img.getNumber('SUN_ELEVATION'))
  # 计算地形阴影
  shaded_relief = ee.Terrain.hillShadow(elevation, azimuth, zenith, 200, True)
  return img.updateMask(shaded_relief)

In [ ]:
elevation = ee.Image('CGIAR/SRTM90_V4')

In [ ]:
#计算不同的水体指数
def WIs (image, roi):
    image = image.clip(roi)
    #提取波段
    """ blue = image.select('SR_B2')
    green = image.select('SR_B3')
    red = image.select('SR_B4')
    nir = image.select('SR_B5')
    swir1 = image.select('SR_B6')
    swir2 = image.select('SR_B7') """
    #计算水体指数
    #归一化水体指数，NDWI =(G-NIR)/(G+NIR)
    ndwi=image.normalizedDifference(['SR_B3', 'SR_B5']).rename("NDWI")
    #改进的归一化水体指数，MNDWI=(G-SWIR)/(G+SWIR)
    mndwi=image.normalizedDifference(['SR_B3', 'SR_B6']).rename("mNDWI")
    
    awei0 = image.expression('0.0004*(green-swir1) - 0.000025*nir - 0.000275*swir2',
        {
          'green': image.select('SR_B3'),
          'swir1': image.select('SR_B6'),    
          'nir': image.select('SR_B5'),    
          'swir2': image.select('SR_B7'),
        }).rename('awei0')
    
    awei1 = image.expression('0.0001*blue+0.00025*green-0.00015*(nir+swir1)-0.000025*swir2',
                             {
                                'blue': image.select('SR_B2'),
                                'green': image.select('SR_B3'),
                                'nir': image.select('SR_B5'),
                                'swir1': image.select('SR_B6'), 
                                'swir2': image.select('SR_B7'),
                             }).rename('awei1')
    
    wi2015 = image.expression('1.7204 + 0.0171*green+0.0003*red-0.0070*nir-0.0045*swir1-0.0071*swir2',
                              {
                                'green': image.select('SR_B3'),
                                'red': image.select('SR_B4'),
                                'nir': image.select('SR_B5'),
                                'swir1': image.select('SR_B6'), 
                                'swir2': image.select('SR_B7'),
                              }).rename('wi2015')
    wiThrd = [-0.21, 0, -0.07, -0.02, 0.63]
    wtWght = [0.000, 0.640, 0.008, 0.019, 0.333]
    cdwiThrd = 0.648
    wis = [ndwi, mndwi, awei0, awei1, wi2015]
    fns = ['ndwi', 'mndwi', 'awei0', 'awei1', 'wi2015']
    water_images = []
    for i in range(5):
        wt = wis[i]
        threshold = wiThrd[i]
        water = wt.gte(threshold).rename(fns[i] + '_water')
        water_images.append(water)
    cdwi = (water_images[0].multiply(wtWght[0])
            .add(water_images[1].multiply(wtWght[1]))
            .add(water_images[2].multiply(wtWght[2]))
            .add(water_images[3].multiply(wtWght[3]))
            .add(water_images[4].multiply(wtWght[4])))
    cdwiWt = cdwi.gte(cdwiThrd).rename('CDWI_water').clip(roi)
    return cdwiWt
        

In [ ]:
landsat8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
            .filterBounds(roi) \
            .filterDate("2020-01-01", "2020-12-30")\
            .map(maskL8sr)\
            .map(lambda image: hillshade(image, elevation))\
            .map(lambda image: WIs(image, roi))\
            .mean()

In [ ]:
Jrs.bandNames().getInfo()

In [ ]:
waterThreshold = 0.25
waterMask = landsat8.clip(roi).gt(waterThreshold)
Map.addLayer(waterMask, {}, 'CDWI Water Mask')
Map

In [ ]:
bands = waterMask.select(['CDWI_water']) 


In [ ]:
waterMask.bandNames().getInfo()

In [ ]:
Jrs = ee.ImageCollection("JRC/GSW1_4/YearlyHistory") \
            .filterBounds(roi) \
            .filterDate("2020-01-01", "2020-12-31")\
            .median()
visualization = {
  'bands': ['waterClass'],
  'min': 0.0,
  'max': 3.0,
  'palette': ['cccccc', 'ffffff', '99d9ea', '0000ff']
}
Map.addLayer(Jrs.clip(roi), visualization, 'Jrs')
Map


In [ ]:
jrcMask = Jrs.eq(1)
correctedData = waterMask.where(jrcMask, 0)
type(correctedData)
Map.addLayer(correctedData,{},'correctedData')
#Map.addLayer(jrcMask,{},'jrcMask')
Map

In [ ]:
geemap.download_ee_image(correctedData, r"E:\湿地.tif", region=roi, crs='epsg:4326', scale=30)

In [ ]:
geemap.ee_export_image_to_drive(correctedData,  description="landsat", fileNamePrefix='湿地', folder="湿地", region=roi, scale=30, crs='epsg:4326', maxPixels=1e11)

In [ ]:
# 重采样
resampled = correctedData.reproject(crs=correctedData.projection().crs(), scale=1000) \
                        .reduceResolution(reducer=ee.Reducer.mean(), bestEffort=True)
#reprojected = correctedData.resample('bicubic')
Map.addLayer(resampled,{},'resampled')
Map
